<a href="https://colab.research.google.com/github/parkyena2/googleColab/blob/main/dataScienceIntroduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [ ]:
pip install yahoo_fin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 9.1 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=a589deab76d71b741cadcc990551dfd9efb38292e4fd93522019b364a39bcd12
  Stored in directory: /root/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24589 sha256=edf2dc5e7c32ec884c5e11793af5aa94c247262e933c160abe7922a00d52ef5e
  Stored in directory: /root/.cache/pip/wheels/d6/9c/58/ee

In [ ]:
import yahoo_fin.stock_info as si
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython
from datetime import datetime
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pickle
from pandas.plotting import scatter_matrix
from dateutil.relativedelta import relativedelta
from sklearn.decomposition import PCA

In [ ]:
ticker = "googl"
# get_earnings_history(기업) = 기업 분기별 이익 불러오는 함수
earning_list = si.get_earnings_history(ticker)

SyntaxError: ignored

In [ ]:
# get_earnings_history(기업) 반환값 = 리스트 형태
# type() 명령어 = 어떤 데이터(변수)의 타입 검사
type(earning_list)

In [ ]:
# 파이썬 인덱스 = 0부터 시작
# :2 = 0 ~ 2의 바로 앞까지 범위 = 0, 1
earning_list[:2]

* companyshortname = 회사 이름
  * (Alphabet Inc = 구글 대주주)
* epsactual = 실제 해당 분기에 발생한 이익 값
  * (None = 관측되지 않음)
  * (아직 발표되지 않음)
* epsestimate = 해당 분기 발생 이익 값 예측
  * (epsactual 발표 전 분석가들이 추측한 값)
* epssurprisepct = 추측 값(epsetimate)과 비교해 실제 관측 값(epsactual)이 보인 차이(percent)
  * (실제 관측 값 epsactual이 None이므로 관측 불가 = 이 값도 None)

# **Pandas를 이용한 데이터 가공**

# 데이터 프레임 생성

## 1. 리스트로부터 데이터 프레임 생성

In [ ]:
# 리스트로부터 데이터 프레임 생성
# pd.DataFrame() 함수의 parameter로 data=earning_list를 넘겨 줌
# 반환값 -> earning_df에 저장
earning_df = pd.DataFrame(data=earning_list)

NameError: ignored

In [ ]:
# head() = 데이터 프레임의 시작부를 보여 줌
earning_df.head()

NameError: ignored

데이터 프레임 구조:
<br>
* 최상단에 column 이름들 위치
* dictionary -> 데이터 생성했으므로
  * key = column 이름
  * value = column 값
* 좌측에 0부터 시작하는 index 매겨짐
  * index 이용해 데이터 선택 가능

In [ ]:
# dataFrame.to_csv('path', index=저장여부)
# (가공한) 데이터 프레임을 csv 파일로 저장
# pandas 제공
earning_df.to_csv('earning.csv', index=False)

* csv 파일 = 값들을 콤마로 분리한 파일 형태
  * 흔히 생각하는 엑셀 파일과 유사한 텍스트 파일 형식
* to_csv() 함수의 parameter로 index=False 옵션을 주지 않으면 인덱스 값까지 csv 파일에 저장됨
  * 인덱스 값은 의미 있는 데이터 값이 아닌
  * 데이터 구조 생성 시 자동으로 매겨지는 값이므로
  * 굳이 파일에 저장할 필요 없음 = 일반적으로 저장 안함

## 2. csv 파일로부터 데이터 프레임 생성

In [ ]:
# 주어진 csv 파일을 읽어 들여 데이터 프레임 생성
# pd.read_csv('path')
earning_df_from_csv = pd.read_csv('earning.csv')
earning_df_from_csv.head()

* 값을 읽어 올 때 기본 구분자seperator = 콤마(,)
  * sep=","
  * 다른 구분자를 쓸 경우 parameter로 sep="구분자" 사용
* 앞서 저장할 때 None이었던 값 -> NaN으로 바뀌어 저장된 것을 확인할 수 있음
* 엑셀 파일도 읽어 올 수 있음
  * pd.read_excel('path')

## 3. NumPy 배열로부터 데이터 프레임 생성

### 데이터 프레임 값들을 NumPy 배열로 추출

* NumPy 배열로부터 데이터 프레임 생성에 앞서
* NumPy 배열 생성

In [ ]:
# .values attribute로 컬럼명 제외한 값들만 반환받아 변수 earning_array에 저장
earning_array = earning_df.values
# earning_array 범위 지정
earning_array[:2]

* NumPy = 데이터를 배열로 다루는 라이브러리
  * 데이터 프레임 값들을 NumPy 배열로 추출하기 앞서
  * 데이터 프레임을 배열로 변환해야 함
* .values attribute
  * 객체의 values 속성(attribute)에 접근
  * 타 언어에서는 attribute -> property라고도 함
* array 범위 지정하는 방법: 일차원적으로 파이썬 list와 동일
  * 대괄호 [ ]와 콜론 : 사용해 지정 가능
* 출력 결과 values attribute로 저장한 값이 일종의 array임을 알 수 있음
  * 단순한 값들의 나열이므로.
  * 데이터 프레임 = 2차원 배열 형태였으므로 nest된 배열의 형태
    * 리스트에 리스트가 nest된 이중 리스트

### 만들어진 NumPy 배열로부터 데이터 프레임 생성

In [ ]:
# 사용 함수 = pd.DataFrame() -> 앞과 동일
# NumPy 배열에는 column 이름이 없으므로 columns= 파라미터 사용해 열 이름 지정
  # 데이터 프레임에서 열 이름 = dataFrame.columns.values
earning_df_from_array = pd.DataFrame(data=earning_array, columns=earning_df.columns.values)
# 테스트 출력
earning_df_from_array.head()

# 데이터 선택과 샘플링

## 데이터 프레임의 특정 열 column 선택



In [ ]:
earning_cols = earning_df[["startdatetime", "epsactual"]]
earning_cols.head()

* 두 개의 열 startdatetime, epsactual 선택
  * dataFrame[지정 조건] 형태로 특정 범위 기본 선택:
  * 원하는 column들 이름을 리스트 형태로 제공

### 시리즈 Series

In [ ]:
earning_df["epsactual"]

* 열을 한 개만 지정한 경우 return되는 데이터 타입 = 시리즈Series
  * 일차원 배열 형태의 데이터 타입
  * 좌측 숫자 = 인덱스

In [ ]:
type(earning_df["epsactual"]), type(earning_df[["startdatetime", "epsactual"]])

* 데이터 프레임에서
  * 한 개 열만 선택한 경우:
    * 반환 데이터 타입 = 시리즈
  * 두 개 이상 열 선택한 경우:
    * 반환 데이터 타입 = 데이터 프레임

## 데이터 프레임의 특정 행 row 선택

* 파이썬 list에서의 범위 지정과 유사하게 동작

### 암묵적 implicit 데이터 선택

* index 값에 기반하는 것이 아닌
* 데이터 프레임의 행 순서에 기반

In [ ]:
[1, 2, 3, 4, 5][1:3]

* 1, 2, 3, 4, 5를 원소로 가진 list의 범위를
* 1:3으로 지정

In [ ]:
earning_rows = earning_df[1:3]
earning_rows

* earning_df 데이터 프레임의 1:3 범위 행 선택
  * 1, 2행 return

In [ ]:
earning_df_new_index = pd.DataFrame(
    data=earning_array,
    columns=earning_df.columns.values,
    index=range(1, len(earning_df) + 1)
)
earning_df_new_index.head()

* index 값을 1부터 시작하도록 새로 매겨도

In [ ]:
earning_df_new_index[1:3]

* 1, 2번 인덱스를 선택하지 않고
* 여전히 0번째부터 세어 1, 2번째 행 출력
<br>

-> **암묵적 데이터 선택**

#### iloc을 이용한 데이터 선택

* iloc = implicit location
* 대괄호를 사용한 방식과 정확히 일치하는 암묵적 데이터 선택 방법

In [ ]:
earning_df_new_index.iloc[1:3]

### 명시적 데이터 선택

* index에 기반한 데이터 선택

#### loc를 사용한 데이터 선택

In [ ]:
earning_df_new_index.loc[1:3]

* 지정한 인덱스에 따라 데이터 선택
* 콜론 : 범위 지정 방식도 파이썬에서의 일반적 사용 방식과 다름
  * 1:3 = 1, 2, 3 index 모두를 선택

## 데이터 프레임에서 행과 열 범위 동시 지정해 선택

In [ ]:
earning_df_new_index.iloc[1:4,[2,5]]

In [ ]:
earning_df_new_index.loc[1:3,["startdatetime", "epsactual"]]

## 조건에 의한 데이터 선택

In [ ]:
earning_df[earning_df["epsactual"] > 20]

### 두 개 이상 조건 사용하기

* 각각의 조건을 소괄호로 묶고 &로 연결

In [ ]:
earning_df[(earning_df["epsactual"] > 2) &
           (earning_df["startdatetime"] < "2021-01-01")]

### 조건에 맞는 인덱스 추출: np.where()

In [ ]:
index = np.where((earning_df["epsactual"] > 20) &
                 (earning_df["startdatetime"] > "2021-01-01"))
index

In [ ]:
earning_df.loc[index]

### sample(): 데이터 프레임의 행들을 랜덤 추출

In [ ]:
len(earning_df.sample(n=5))

* 샘플 수 -> n= 값으로 지정
  * n=5 -> 5개 행만 랜덤 추출
* len = 데이터 길이를 반환하는 파이썬 함수

In [ ]:
len(earning_df.sample(frac=0.5)), len(earning_df)

* frac = fraction(비율)
  * 전체 = 1로 봤을 때의 비율을 parameter로 사용
  * frac(0.5) = 0.5 만큼의 비율 = 전체의 약 50%

## earning_df에서 실제로 분석에 사용될 열만 선택하기

In [ ]:
earning_df = earning_df[["startdatetime",
                         "epsactual",
                         "epssurprisepct"]]
earning_df.head()

# 누락 데이터의 처리

* 데이터 누락missing = 관측 값이 존재하지 않는 경우
* 데이터 프레임에서 NaN으로 표시
  * 데이터의 특정 값이 NaN으로 표기돼 있을 시 연산 적용이 불가능한 경우 존재: 누락 데이터 처리 필요
* 누락된 데이터 채움 = **imputation**
  * 기본값 = 디폴트값 = 0 등으로 채우기
  * 해당 열의 평균 또는 중간 값으로 채우기
  * NaN이 존재하는 행의 다른 값들로 NaN의 값을 추정하는 모델을 만들어 모델의 예측 값으로 채우기

## fillna(): NaN인 부분을 주어진 값으로 채우기

In [ ]:
earning_fillna = earning_df.fillna(
    {'epsactual': 30.69,
     'epssurprisepct': 12.25})
earning_fillna.head()

* fillna의 parameter로 dictionary 값 전달
  * key(column명)에 해당하는 value(NaN 값을 대체할 값) 전달

In [ ]:
earning_df.fillna?

## dropna(): NaN 값 포함한 행들 삭제

* 다루는 데이터가 대량일 경우(충분히 많은 경우) 일부 NaN 값이 존재하는 행을 삭제해도 문제되지 않을 수 있음
* dataFrame.dropna() 함수로 NaN 값 포함하는 행 모두 삭제하고 NaN 값 없는 데이터만 반환 받아 사용

In [ ]:
earning_sample = earning_df.dropna()
len(earning_sample), len(earning_df)

In [ ]:
earning_sample.head()

## 인덱스의 재설정: reset_index()

* 기존 인덱스를 무시하고 0부터 1씩 증가하는 인덱스를 재설정
* dropna() 함수 등으로 특정 행 삭제 후 공백이 생기게 되는 인덱스를 재설정할 때 주로 사용

In [ ]:
earning_sample = earning_sample.reset_index(drop=True)
earning_sample.head()

* drop=True 옵션이 없으면 (drop=False)
  * 기존 인덱스 값 -> 새로운 컬럼으로 생성
* 일반적으로 인덱스값은 컬럼으로 보존할 만큼 유의미한 정보를 주는 데이터가 아니기 때문에
  * 기존 인덱스 값을 버리고 새로운 인덱스 값만 남김

# 중복 데이터의 제거

* 같은 값이 중복될 경우 제거하는 편이 좋음
  * 서로 다른 행이 중복 = 하나의 객체에 대한 인스턴스가 여러 번 관측, 기록됐을 가능성 존재
  * 이와 같이 오류로 인한 중복 데이터를 모델 생성 또는 분석에 사용 시 문제 발생 가능

## 중복 데이터 생성

* 현재 데이터에 중복 데이터가 존재하지 않으므로
* 중복 데이터 제거 예시를 위해 임의로 중복 데이터 생성

### pd.concat(): 데이터 붙이기

In [ ]:
earning_dup = pd.concat([earning_sample[:1],
                         earning_sample]).reset_index(drop=True)
earning_dup.head()

* pd.concat() 함수의 parameter로 붙일 데이터 프레임의 리스트를 전달
1. earning_sample 데이터 프레임의 0번째 행을
2. earning_sample 데이터 프레임과 붙여
3. 0번째 행이 중복되는 데이터 프레임 생성
4. reset_index() 함수로 인덱스 재설정

## drop_duplicates(): 중복 데이터 제거

In [ ]:
earning_dup.drop_duplicates(subset=["startdatetime"],
                            keep="first", inplace=True)
earning_dup.head()

* 행의 컬럼 전체가 중복되는 경우가 아닌
* 특정 컬럼 값들만 중복되는 경우도 중복 데이터로 간주할 수 있음
  * subset= 파라미터로 지정 -> 특정 컬럼 부분집합만 겹쳐도 중복 행으로 간주
* keep= 파라미터:
  * 중복 행 삭제 시 남길 행 선택
  * first, last 등
* **inplace= 파라미터:**
  * 데이터 프레임 가공 시 매우 많이 등장하는 옵션
  * inplace=False
    * 함수를 적용한 결과 데이터 프레임을 반환하고
    * 반환된 데이터 프레임을 새 변수명으로 저장
    * 이 경우 기존 데이터 프레임, 반환된 데이터 프레임 모두 메모리 공간을 차지하게 됨 = 낭비 심함
  * inplace=True
    * 함수 적용 결과 반환된 데이터 프레임이 기존 데이터 프레임 대체
    * 즉: 함수가 대상 데이터 프레임 자체를 변형
    * 결과적으로 컴퓨터 메모리 절약

# 데이터 연산

## split() 함수: 문자열 자르기

In [ ]:
earning_sample.dtypes

In [ ]:
"2021-10-26T00:00:00.000Z".split("T")

In [ ]:
"2021-10-26T00:00:00.000Z".split("T")[0]

## apply(): 데이터 프레임의 모든 행 / 열에 함수 적용

* axis=1 -> 모든 행에 함수 적용
* axis=0 -> 모든 열에 함수 적용

In [ ]:
earning_sample["date"] = earning_sample.apply(
    lambda row: row["startdatetime"].split("T")[0], axis=1)
earning_sample.head()

* lambda 함수 = 익명함수, 무명함수
  * lambda = 굳이 이름을 부여할 필요 없는 일회성 함수 선언 위한 키워드
  * lambda *parameter*: *command*

In [ ]:
earning_sample["date"] = pd.to_datetime(earning_sample["date"],
                                        format="%Y-%m-%d")

In [ ]:
earning_sample.dtypes

* apply(), split() 함수로 분리한 "date" 컬럼의 데이터 포맷 변경

### apply(): 데이터 프레임 뿐만 아니라 Series 타입에도 적용 가능

* 이 경우 axis 필요 없음

In [ ]:
earning_sample["startdatetime"].apply(
    lambda startdatetime: startdatetime.split("T")[0])

## map(): Series의 모든 값들에 대해 함수 적용

* apply()와 차이:
  * apply() = 데이터 프레임, 시리즈에 모두 사용 가능
  * map() = 시리즈에 사용 가능

In [ ]:
earning_sample["startdatetime"].map(
    lambda startdatetime: startdatetime.split("T")[0])

## applymap(): 데이터 프레임 안 모든 값들에 함수 적용

In [ ]:
(pd.DataFrame({"a": [1, 2], "b": [3, 4]})
  .applymap(lambda x: x + 1))

* 이 경우 lambda의 parameter = 데이터 프레임의 개별 값들
  * apply()에서의 lambda parameter = 데이터 프레임의 행들

# **NumPy를 이용한 데이터 가공**

# 구글의 주가 데이터 불러오기

In [ ]:
price = si.get_data(ticker)
type(price)

* earning(list 타입)과 달리 데이터 프레임 타입
  * 데이터 프레임으로 변환 및 생성 과정 없이 바로 가공 가능

In [ ]:
type?

In [ ]:
price.head()

* 인덱스 = 해당 주가의 날짜
* open = 그날 시가 (시작가)
* close = 그날 종가 (종료가)
* high = 그날 가장 높게 거래된 가격
* low = 그날 가장 낮게 거래된 가격
* volume = 거래량
* ticker = 구글 (주식회사)

# 데이터 정렬

* 오름차순 정렬: 데이터가 커지는 순서대로 정렬
* 내림차순 정렬: 데이터가 작아지는 순서대로 정렬

## sort_index(): 인덱스 기준 정렬

In [ ]:
price_sort_index = price.sort_index(ascending=False)
price_sort_index.head()

* 최근부터 과거 날짜로 내림차순 정렬: ascending=False

## sort_values(): 값을 기준으로 정렬

In [ ]:
price_sort_values = price.sort_values(by='close', ascending=False)
price_sort_values.head()

* by='정렬기준 컬럼'
* ascending=False: 내림차순 정렬

# 관심 질문 수립하기

데이터 분석 = 관심있는 질문의 답을 구하는 것
1. 분석 전에 질문을 수립하고
2. 질문에 대한 답을 구하기 위해
3. 데이터를 어떻게 수집하고 분석할지 계획하는 것이 중요

분석 중에도
* 데이터 수집단계(시작)부터 계획을 세우는 데 관여할 수 있는 실험과 분석이 가능한 경우가 있고
* **이미 발생한 데이터 관측값들에 대해 관심 질문을 던지고 답을 구하는** - 관찰 연구에 해당하는 분석이 있음

관심 질문:
* (구글의) 주당순이익(EPS, earning per share)과 주가는 선형적 관계를 갖는가?
  * 주당순이익: 기업의 주식 한 주당 기업이 버는 이익
    * 벌어들인 이익 ÷ 총 주식 수 = EPS (이익/주)
  * 선형적 관계: 주당순이익이 증가하면 주가도 그에 정비례해 증가하는지?

관심 질문에 답하기 위해:
* **주당순이익이 발표된 달의 다음 한 달 동안 종가 평균**(주가)을 구해서 비교해보기
  * 주당순이익은 이미 주어져 있으므로 주당순이익과 비교할 주가 정의 필요
    * 주당순이익이 발표된 당일 또는 다음날 주가를 사용할지?
    * 시가와 종가 중 어느 것을 사용할지?
  * 주당순이익이 발표되고 - 발표된 정보가 주식 시장에 반영되는 데에 시간이 필요하다는 가정으로 주가 정의
* 데이터를 월별로 묶어 평균 구해야 함
  * 매달 종가 평균 값을 구해야 하므로
  * 종가값들을 월별로 묶어 월별 평균 구해야 함
* groupby와 agg 함수 (aggregate: 집계) 사용
  * 월별 그룹화 = groupby 함수
  * 평균 구하기(집계) = agg 함수

# 그룹과 집계

## month 컬럼 생성

In [ ]:
price["month"] = price.index.map(
    lambda date: date.strftime("%Y") + "-" + date.strftime("%m"))
price.head()

## groupby() & agg(): 월별 그룹화해 시가평균, 종가평균 구하기

In [ ]:
price_mean = (price
              .groupby(by="month")[["open", "close"]]
              .agg(open_mean=("open", "mean"),
                   close_mean=("close", "mean"))
              .reset_index())
price_mean.head()

* groupby(by="month")[["open", "close"]]
  * month 컬럼 **기준**으로
  * open, close 컬럼을 그룹화
* agg( : 집계 함수 적용
  * open_mean=("open", "mean"),
    * 시가 평균 산출:
    1. open 컬럼에 대해
    2. mean 함수 적용 후
    3. 새로운 컬럼 open_mean 으로 표시
  * close_mean=("close", "mean"))
    * 종가 평균 산출
    1. close 컬럼에 대해
    2. mean 함수 적용 후
    3. 새로운 컬럼 close_mean 으로 표시
* reset_index()
  * 인덱스 값 재설정

-> 결과 월단위로 그룹화된 시가와 종가에 대해 평균 산출됨

### agg에 딕셔너리를 넘기는 경우

In [ ]:
price_mean = (price
              .groupby(by="month")
              .agg({"open": "mean", "close": "mean"})
              .reset_index())
price_mean.head()

* 이전 코드와 같은 집계, 다른 문법
* 새로운 이름의 컬럼을 생성하지 않고
* 기존 컬럼과 같은 컬럼명으로 집계한 평균값 표시
* agg() 함수의 파라미터로 딕셔너리 전달
  * key = 집계 대상 컬럼
  * value = 컬럼에 적용할 집계 함수

### rename(): axis=1 옵션 사용해 열 이름 변경

In [ ]:
price_mean.rename(
    {"open": "open_mean", "close": "close_mean"},
    axis=1,
    inplace=True)
price_mean.head()

* 데이터 프레임 다룰 때 컬럼 이름 변경하는 연산이 흔함
* 인자 1 = 딕셔너리
  * key = 기존 컬럼명
  * value = 바꿀 컬럼명
* axis=1 옵션을 사용해 열 이름 변경
* inplace=True
  * 변경한 내용을 원본 데이터 프레임에 그대로 적용

### agg에 인자로 줄 수 있는 함수들:

* mean = 평균
* sum = 합계
* min = 최소값
* max = 최대값
* median = 중간값
* std = 표준편차
* var = 분산
* count = 그룹에 속한 샘플 수

In [ ]:
(price
 .groupby(by="month")
 .agg({"open": "min", "close": "max"})
 .reset_index()
 .head())

## agg() 없이 바로 집계함수 적용하기

In [ ]:
(price
 .groupby(by="month")[["open", "close"]]
 .mean()
 .reset_index()
 .head())

In [ ]:
(price
 .groupby(by="month")[["open", "close"]]
 .max()
 .reset_index()
 .head())

## reset_index() 호출하지 않는 경우: 인덱스 = groupby()에 사용된 값

In [ ]:
price.groupby(by="month")[["open", "close"]].mean().head()

# 시계열 데이터의 처리

* 특정 분기의 주당순이익이 아닌
* 지난 1년간 주당순이익 합 사용
  * 특정 분기에 따른 **계절적 영향 배제**하고 기업 이익 고려
* 분기당 1번씩 주당순이익 발표
  * 최근 4개의 주당순이익 값을 한 칸씩 이동하며 계산

* 특정 크기 윈도를 한 칸씩 이동시키며 원하는 계산 적용
  * 시계열 데이터에서 자주 필요할 수 있음

-> **Pandas의 rolling() 함수로 수행**

## rolling(): 윈도window를 이동시키며 함수 적용

In [ ]:
roll = earning_sample["epsactual"].rolling(window=4)
earning_sample["eps_ttm"] = roll.sum().shift(-3)
earning_sample.head()

* earning 데이터의 epsactual = 실제 해당 분기에 발표된 주당순이익 값
* eps_ttm = 1년치 epsactual 값의 합
  * 해당 행부터 시작해 4개의 epsactual 값을 합한 값
  * ttm = Trailing to Twelve Months

In [ ]:
roll.sum().head()

* shift(-3) 하는 이유:
  * roll.sum()은 4개 epsactual 값의 합이므로 0행이 아닌 3행부터 저장됨
  * 0~3행은 NaN 값이 되므로
  * 위로 3칸 끌어 올려 줌

## differencing: 인접한 두 값의 차이 계산

* filtering이라고도 함
* 시계열 데이터에 흔히 적용하는 연산 중 하나
  * 시계열 데이터에 존재하는 계절성을 없애 용이한 모델링 수행할 수 있는 경우가 있음
* 두 값 사이 차를 계산한 결과가 새로운 정보를 주는 경우 있음

In [ ]:
earning_sample["eps_ttm_diff"] = (
    earning_sample["eps_ttm"]
    .rolling(window=2)
    .apply(np.diff)
    .shift(-1) * (-1))
earning_sample.head()

* rolling(window=2)로 생성된 2칸짜리 윈도에 대해
* apply(np.diff) 연산으로 differencing 수행
* shift(-1)로 한 행 위로 끌어 올려 줌
* 뒤쪽 값에서 앞쪽 값을 빼기 때문에 음수값 나옴 -> (-1) 곱해 줌

# 데이터 프레임의 병합

## merge: 데이터 프레임 병합

* 데이터베이스 테이블의 조인join에 해당하는 연산
* 서로 다른 테이블에 존재하는 값들의 비교, 또는 묶어 모델을 생성하기 위해 테이블들을 합칠 필요가 있음
* 각각의 테이블에 존재하는 서로 공통되는 값들을 기준으로 병합 가능

In [ ]:
earning_sample["month"] = (
    earning_sample["date"]
    .map(lambda date:
         ((date + relativedelta(months=1)).strftime("%Y") +
          "-" +
          (date + relativedelta(months=1)).strftime("%m"))))
earning_sample.head()

* merge할 테이블과 공통 컬럼 = month 생성

In [ ]:
earning_sample = (
    earning_sample[(earning_sample["date"] >= "2015-01-01") &
                   (earning_sample["date"] <= "2019-12-31")])
earning_sample.head()

* 기존 데이터에서 날짜 기준으로 범위 한정해 일부만 추출
  * 분석 시 교재와 같은 결과를 보기 위함

### merge 방식 1: how="left"

두 데이터 프레임 중 왼쪽 인자에 해당하는 데이터 프레임 기준으로 병합
* 왼쪽 데이터 프레임의 값들은 모두 살리고
* 오른쪽 데이터 프레임의 값들 중 왼쪽 데이터 프레임의 month값과 일치하는 행들만 가져옴
* 왼쪽 데이터 프레임의 month 값과 일치하는 오른쪽 데이터 프레임의 행이 없으면 NaN 값으로 채움

In [ ]:
price_earning = pd.merge(
    earning_sample, price_mean, on="month", how="left")
price_earning.head()

* pd.merge(왼쪽df, 오른쪽df, 기준컬럼, merge방식)

### merge 방식 2: how="right"

In [ ]:
pd.merge(
    earning_sample,
    price_mean,
    on="month",
    how="right").head()

### merge 방식 3: how="inner"

* 두 데이터 프레임에 공통적으로 존재하는 값들(값들의 교집합)을 키로 병합 수행

In [ ]:
set(earning_sample["month"]).issubset(set(price_mean["month"]))

In [ ]:
(len(pd.merge(earning_sample, price_mean,
              on="month", how="left")) ==
 len(pd.merge(earning_sample, price_mean,
              on="month", how="inner")))

### merge 방식 4: how="outer"

* 두 데이터 프레임에 존재하는 값들의 합집합을 키로 병합 수행

# 데이터 분석

* 간단한 회귀분석 모델 만들기

## 분석에 사용할 컬럼만 선택

In [ ]:
price_earning = price_earning[["month", "eps_ttm", "eps_ttm_diff",
                               "epssurprisepct", "close_mean"]]
price_earning.head()

## 데이터 프레임 저장

In [ ]:
price_earning.to_csv("price_earning.csv", index=False)

## describe(): 데이터 프레임 각 열에 대한 기초통계 출력

In [ ]:
price_earning.describe()

## 산점도 행렬 scatter plot matrix 만들기: 데이터 프레임의 열 사이 관계를 그래프로 표시

In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

In [ ]:
plt.rcParams.update({'font.size': 14})

In [ ]:
scatter_matrix(price_earning[["eps_ttm", "eps_ttm_diff",
                              "epssurprisepct", "close_mean"]],
               alpha = 0.8, figsize = (8, 8), diagonal="hist")
plt.show()

* month 컬럼을 제외한 컬럼들 사이 관계 표시
* alpha = 투명도
* figsize = 표시될 이미지 크기
* diagonal="hist"
  * 산점도 행렬의 대각선에 히스토그램 표시

## corr(): 상관 계수 correlation 구하기

In [ ]:
corr = price_earning[["eps_ttm", "eps_ttm_diff",
                      "epssurprisepct", "close_mean"]].corr()
corr

## 선형 회귀 linear regression 모델

* 변수들 사이에 선형적 관계가 있는지 분석하기 위해 활용하는 모델

### ols(): R과 유사한 형태로 선형 회귀 모델 생성

* ols = ordinary least squares

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

* statsmodels
  * 파이썬에서도 R과 유사한 형태로 선형회귀 모델 생성하기 위해 사용하는 패키지
* 패키지에서 ols 함수 import

In [ ]:
model = ols(
    "close_mean ~ eps_ttm + eps_ttm_diff + epssurprisepct",
    data=price_earning
    ).fit()
model.summary()

* close_mean = y = 반응변수
* eps_ttm, eps_ttm_diff, epssurprisepct = x = 예측변수

-> y와 x 사이에 선형적 관계가 있는지 분석: fit()으로 모델 생성
<br>
-> R에서와 똑같이 summary() 함수 사용해 요약 결과 표시

eps_ttm에 대한 t-test의 p-value가 0에 가까운 값이면 close_mean과 강한 선형적 연관관계 가짐
* 일반적으로 p-value의 의미:
  * 귀무가설이 참이라고 가정했을 때
  * 제시된 t 또는 그보다 극단적인 값을 관측할 확률
* 여기서 귀무가설: coef=0
  * = x들이 y와 선형적 관계 없음
  * 0 가설, null hypothesis라고도 함

-> x와 y가 선형적 관계가 없다면 지금 보는 통계값이나 그보다 극단적인 값을 관측할 확률이 0에 가까움
<br>
-> eps_ttm에 대해 귀무가설이 참이라고 보기 어려움
<br>
-> **eps_ttm에 대해 귀무가설 기각**
<br>
-> 나머지 x들은 귀무가설 기각 어려움 = y와 선형적 관계를 갖는다고 보기 어려움

결론: eps_ttm만이 close_mean과 강한 선형적 관계 갖는다

### eps_ttm만을 예측변수 predictor variable로 포함하는 모델 생성

In [ ]:
model = ols("close_mean ~ eps_ttm", data=price_earning).fit()
model.summary()

* R-squared, Adj. R-squared:
  * x의 값이 y의 변동을 얼마나 설명하는지
  * Adj. R-squred는 x를 많이 사용하면 패널티를 주는 방식이므로 이전보다 약간 증가함
  * 이전 값과 큰 변동이 없음 = eps_ttm만으로도 close_mean의 변동을 충분히 설명할 수 있음

### 잔차 residual 분석

* ols()를 사용해 만든 선형회귀 모델은 아무 상황에나 적용할 수 있는 것은 아님
* 주어진 데이터가 어떤 가정들을 만족할 때 적용 가능

-> 잔차분석을 통해 선형회귀 모델의 가정들을 데이터가 만족하는지 분석 가능
<br>
-> 이를 위해 residual plot 그려야 함

residual 정의:
* y의 actual value - y의 fitted value(어떤 x에 대해 모델이 y를 예측한 값 = x)

In [ ]:
pickle.dump(model, open("model.pickle", "wb"))

In [ ]:
# x = model에서 fittedvalues에 액세스해 얻어 옴
fitted = model.fittedvalues
# y = residual 값들 얻어 옴
resid = model.resid
fig, ax = plt.subplots(figsize=(10, 7))
# scatter plot 그리기
ax.scatter(fitted, resid)
# y=0 선 긋기
ax.axhline(y=0, color='black', linestyle='solid')
ax.set(xlabel="Fitted Value", ylabel="Residual", title="Residual vs Fits Plot")
plt.show()

y=0을 중심으로 점들이 비슷한 간격으로 분산돼 있으면:
<br>
회귀분석의 조건 중
* 등분산성 equality of variance과
* 선형성 linearity 만족

<-> 반대로 점들이 균일하게 퍼져있지 않고 강한 패턴이나 증감하는 경향 등 규칙성을 보일 때: 이러한 가정을 위반함

In [ ]:
# resid 값 얻어 오기
residuals = model.resid
# probability plot 생성
pplot = sm.ProbPlot(residuals)
fig = pplot.ppplot(line="45")
plt.show()

회귀분석 조건 중 정규성normality 확인: normal probability plot을 통해 가능
* residual 점들이 정규분포를 따른다고 가정했을 때의 라인에 점들이 가깝게 위치해 있으면 정규성 가정 만족

## 결론

* 간단하게 가공한 데이터로 회귀분석 모델을 만들고
* 구글의 주당순이익(1년간 주당순이익의 합)이 다음 주당순이익이 발표된 다음달의 종가평균과 선형적 관계를 갖는가?

-> 선형회귀 모델 생성:
* 선형적 관계 갖지 않음

## 주의

* 모델 생성 시 시장 금리 등 외부적 변수들을 고려하지 않고
* 기간도 5년 정도로 한정
* 일부 변수만으로 매우 단순한 모델을 생성해 질문에 답하기 위한 분석을 했으므로
* 엄밀히 말하면 이러한 법칙이 성립한다고 보기 어려움

# numpy 라이브러리로 데이터 가공하기

## NumPy란:

* Pandas = 데이터를 데이터베이스 테이블과 유사한 데이터 프레임에 담아 다룰 수 있게 해 주는 라이브러리인 반면
* NumPy = 데이터를 행렬, 벡터로 표현하는 배열에 담아 다룰 수 있게 해 주는 라이브러리
  * 1차원 = 벡터
  * 2차원 = 행렬
  * 3차원 이상 = 배열 (tensor)
  * **선형대수 linear algebra 관련 연산 가능**

## NumPy 배열의 생성

In [ ]:
arr = np.array([[1, 2],
                [2, 1]])

* 리스트들의 리스트로 이차원 배열 표현해 직접 생성

In [ ]:
X = price_earning[["eps_ttm",
                   "eps_ttm_diff",
                   "epssurprisepct"]].values
y = price_earning[["close_mean"]].values
type(X), type(y)

* 데이터 프레임의 values attribute에 접근해 얻어 냄
* ndarray = n dimentional array

## .shape: 배열의 크기 구하기

* 행렬 곱 등 선형대수 연산 시 배열의 크기를 알아야 하는 경우 존재

In [ ]:
arr.shape, X.shape, y.shape

## transpose(): 전치행렬 구하기

In [ ]:
X_t = X.transpose()
X_t.shape

## 배열의 합과 차: 원소들의 합과 차로 계산

In [ ]:
arr + np.array([[1, 0],
                [0,1]])

In [ ]:
arr - np.array([[1, 0],
               [0, 1]])

## 대응되는 원소들끼리의 곱과 나누기

In [ ]:
arr * np.array([[1, 0],
                [0, 1]])

In [ ]:
arr / np.array([[2, 1],
                [1, 2]])

## matmul(): 행렬의 곱

In [ ]:
np.matmul(arr, np.array([[1, 0],
                         [0, 1]]))

## numpy 연산으로 직접 statsmodel의 파라미터 계산해보기

### (𝑿ᵀ𝐗)^(-1)𝐗ᵀ𝐲

### np.ones(): 모든 원소가 1로만 이루어진 배열 만들기

In [ ]:
np.ones((5, 1))

* 파라미터로 shape 지정

### np.concatenate(): 배열에 배열 붙이기

In [ ]:
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
X

1. 인자 1-1: 붙일 배열을 np.ones()로 생성
2. 인자 1-2: 붙일 배열 X
3. 인자 2: axis=1 -> 열과 열처럼 붙임

### X의 전치행렬과 역행렬, 행렬 곱을 이용해 coef 계산

In [ ]:
X_t = X.transpose()
X_t_X = np.matmul(X_t, X)
X_t_X_inv = np.linalg.inv(X_t_X)
X_t_X_inv_X_t = np.matmul(X_t_X_inv, X_t)
coef = np.matmul(X_t_X_inv_X_t, y)
coef

* NumPy의 np.linalg 패키지 함수 np.linalg.inv() 함수로 역행렬 구할 수 있음
* ols()로 모델 생성하고 summary()로 출력한 coef와 동일 값 구해짐

### 라이브러리 없이 파라미터를 계산할 줄 알아야 하는 이유:

* 기존 라이브러리에서 지원하지 않는 나만의 아이디어 구현 시 사용
* 기존 라이브러리가 제공하는 추상화에 문제가 있을 경우 디버깅 필요
* 모델이 제대로 동작하지 않을 때 원인을 분석, 파악, 수정해야 함

-> 라이브러리 내부에서 무슨 일이 일어나는지 깊이 이해해야 할 필요 있음
<br>
-> 모델 및 분석 라이브러리를 사용하지 않고 NumPy 등의 기본 연산만 제공하는 라이브러리를 사용해 직접 구현하고 개념들을 깊이 이해해야 함

## 열과 행에 대한 집계

### np.mean, np.var

In [ ]:
X = price_earning[["eps_ttm",
                   "eps_ttm_diff",
                   "epssurprisepct"]].values
np.mean(X, axis=0), np.var(X, axis=0)

* axis=0 -> 열에 대해 함수 적용
* axis=1 -> 행에 대해 함수 적용

In [ ]:
np.mean(X[:, 0]), np.mean(X[:, 1]), np.mean(X[:, 2])

* 범위 지정 시 콜론 : -> 전부 다 

-> 각 열에 대해 mean() 적용한 결과와 axis=0 사용해 mean() 적용한 결과가 동일

## broadcasting

* NumPy의 중요 규칙 중 하나
* 크기가 다른 두 배열 사이에 이항 연산 binary operation이 일어날 때, 특정 규칙 적용해 연산 가능하도록 해 주는 기능

In [ ]:
arr = np.array([[1, 2],
                [2, 1]])
arr + 1

### broadcasting의 활용 - centering: X의 각 원소에서 열의 평균값만큼 차감

In [ ]:
X_mean = np.mean(X, axis=0)
X_mean.shape, X.shape

In [ ]:
X_center = X - X_mean
X_center.shape

* 결과적으로 센터링 수행

### broadcasting의 활용 - 표준화 standardization: 센터링된 X의 각 원소를 표준편차로 나누기

In [ ]:
np.std(X, axis=0)

* 각 열의 표준편차 계산

In [ ]:
A = X_center / np.std(X, axis=0)
A.shape

* 많은 머신러닝, 딥러닝 작업 이전에 표준화 작업 거침

In [ ]:
U, S, V_t = np.linalg.svd(A)
S

In [ ]:
V = V_t.transpose()
V

In [ ]:
transformed = np.matmul(A, V)
transformed[:2]

In [ ]:
pca = PCA(n_components=3)
pca.fit(A)

In [ ]:
pca.singular_values_

In [ ]:
pca.components_

In [ ]:
pca.transform(A)[:2]

# Matplot 라이브러리를 이용해 여러 가지 그래프 그리기

## 데이터 분석 과정에서 시각화의 중요성

* 분석 주제가 되는 질문의 답을 구하기 위해 statistic(통계량)을 사용해 가설을 검정하는 것 외에도
* 시각화를 통한 이해가 데이터 분석의 중요한 한 축 담당:
  * 전반적 경향의 직관적 이해 도움
  * 단일 수치에 의한 판단이 야기할 수 있는 오류를 잡아 줌

-> 가설 검정(기각/안함 두 결과만 존재)보다도 시각화를 통해 좀 더 풍부한 정보

## Matplotlib

* 도표graph 생성을 위한 파이썬의 대표적(기본적) 라이브러리

필수 개념:
* figure: 여러 개의 하위 도표를 포함할 수 있는 캔버스 또는 컨테이너 역할을 하는 객체
* axe: 우리가 실제로 생각하는 개별 도표

-> figure 하나가 여러 개 axe들을 포함할 수 있음

#### pickle 라이브러리

* 파이썬의 메모리 상의 객체를 파일시스템에 저장하거나
* 다시 메모리 상으로 불러올 때 사용
* pandas(텍스트 파일로 내보내고 불러옴)와 달리 바이너리 파일로 내보내고 불러옴

### 불러오기

In [ ]:
import yahoo_fin.stock_info as si
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [ ]:
price_earning = pd.read_csv("price_earning.csv")

## 데이터 정렬

In [ ]:
price_earning.sort_values(by="month",
                          ascending=True,
                          inplace=True)
price_earning.reset_index(drop=True, inplace=True)
price_earning.head()

편의를 위해 과거에서 현재 순으로 데이터 정렬

## scatter() 함수: 산점도 그리기

* axe 객체 = 하나의 도표에 해당하는 역할 담당

In [ ]:
# figure 생성하며 크기 등 기본 사항 정해줄 수 있음
fig = plt.figure(figsize=(10, 5))
# figure 먼저 생성하고 axe 생성 -> axe가 figure에 속한 객체로서 생성됨
ax = plt.axes()
# 실제 그래프 그리기 위해 ax 객체에 scatter() 함수 호출
ax.scatter(price_earning["month"], price_earning["close_mean"])
# set() 함수 호출해 레이블과 타이틀 설정
ax.set(xlabel='month', ylabel='price', title='Google Price')
# set_xticks() 함수 호출해 x축에 표시할 tick들을 지정
ax.set_xticks(price_earning["month"][np.arange(0, len(price_earning), step=3)])
# 플롯 표시
plt.show()

In [ ]:
np.arange(0, len(price_earning), step=3)

### scatter 함수의 파라미터:

In [ ]:
plt.style.use('seaborn-whitegrid')

* 그래프에 격자 추가
* 한 번 plt.style.use() 함수 호출해 스타일 적용 후
* 일일이 그래프 그릴 때 마다 호출하지 않아도 계속 같은 스타일 적용됨


In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.axes()
ax.scatter(price_earning["month"], price_earning["eps_ttm"],
           s=30, c="red", marker="x", alpha=0.5)
ax.set(xlabel='month', ylabel='EPS', title='Google EPS')
ax.set_xticks(price_earning["month"][np.arange(0, len(price_earning), step=3)])
plt.show()

* s: 점의 크기 (size)
* c: 점의 색상 (color)
* marker: 점의 모양
* alpha: 0과 1 사이의 투명도 값

## ⭐ 서브 플롯

* 여러 그래프를 비교, 대조 하기 위해 복수의 도표로 그림 구성
1. plt.subplots() 함수 이용해 figure 객체와 axe 객체들을 얻어 낸 뒤
2. 각각의 axe 객체들 이용해 도표 추가

* subplots() 함수의 nrows, ncols 인자가 도표들이 배치되는 테이블 형태 결정

### 세로 배치

In [ ]:
# axe 객체들 생성하며 순서조 지정
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 14))
ax1.scatter(price_earning["month"], price_earning["close_mean"])
ax1.set(xlabel='month', ylabel='price', title='Google Price')
ax1.set_xticks(price_earning["month"][np.arange(0, len(price_earning), step=3)])
ax2.scatter(price_earning["month"], price_earning["eps_ttm"],
            s=30, c="red", marker="x", alpha=0.5)
ax2.set(xlabel='month', ylabel='EPS', title='Google EPS')
ax2.set_xticks(price_earning["month"][np.arange(0, len(price_earning), step=3)])
plt.show()

### 가로 배치

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 7))
ax1.scatter(price_earning["month"], price_earning["close_mean"])
ax1.set(xlabel='month', ylabel='price', title='Google Price')
ax1.set_xticks(price_earning["month"][np.arange(0, len(price_earning), step=3)])
ax2.scatter(price_earning["month"], price_earning["eps_ttm"],
            s=30, c="red", marker="x", alpha=0.5)
ax2.set(xlabel='month', ylabel='EPS', title='Google EPS')
ax2.set_xticks(price_earning["month"][np.arange(0, len(price_earning), step=3)])
plt.show()

### twinx(): 도표를 겹치게 그리기

* subplots() 함수를 이용한 figure와 axe 객체 생성
  * 한 줄에 모두 생성
* subplots() 함수를 사용하지 않고 figure와 axe 객체를 각각 생성할 수 있음
  * 주석 처리된 부분

-> 둘 모두 동일 결과

In [ ]:
# fig = plt.figure(figsize=(10, 7))
# ax1 = plt.axes()
fig, ax1 = plt.subplots(figsize=(10, 7))
ax1.scatter(price_earning["month"], price_earning["close_mean"],
            label="price")
ax1.set_xlabel("month")
ax1.set_ylabel("price")
ax1.set_title("Google Price and EPS")
ax1.set_xticks(price_earning["month"][np.arange(0, len(price_earning), step=3)])
ax2 = ax1.twinx()
ax2.scatter(price_earning["month"], price_earning["eps_ttm"],
            label="EPS", s=30, c="red", marker="x", alpha=0.5)
ax2.set_ylabel("EPS")
ax2.set_xticks(price_earning["month"][np.arange(0, len(price_earning), step=3)])
fig.legend(loc="upper right")
plt.show()

## 선 그래프

### plot() 함수를 이용한 선 그래프 line plot

* linestyle 파라미터가 선의 종류 결정
  * solid: 실선
  * dashed: dash(-) 기호로 이루어진 선
  * dashdot: dash(-)와 dot(.)이 번갈아 나타남
  * dotted: 점선

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 7))

ax1.plot(price_earning["month"], price_earning["close_mean"],
         label="price", marker="o", c="blue", linestyle="solid")

ax1.set_xlabel("month")
ax1.set_ylabel("price")
ax1.set_title("Google Price and EPS")
ax1.set_xticks(price_earning["month"][np.arange(0, len(price_earning), step=3)])

ax2 = ax1.twinx()

ax2.plot(price_earning["month"], price_earning["eps_ttm"],
         label="EPS", marker="x", c="red", linestyle="dashed")

ax2.set_ylabel("EPS")
ax2.set_xticks(price_earning["month"][np.arange(0, len(price_earning), step=3)])

fig.legend(loc="upper right")

plt.show()

### 수식을 선으로 그리기

* plot() 함수: 데이터 분석에 있어 매우 유용
  * 그래프 분석 시 임의의 선 그어 비교

#### pickle.load(): 이전 저장했던 모델을 로드

In [ ]:
model = pickle.load(open("model.pickle", "rb"))

* 인수 1 = path
* 인수 2 = rb = read binary
  * 읽어 올 파일의 타입 지정 (바이너리)

#### 수식을 이루는 절편과 기울기 불러오기

In [ ]:
model.params

모델의 params 프로퍼티(속성)
<br>
= 모델을 구성하는 parameter들
<br>
= coefficient 값 (회귀분석)
* Intercept 값
  * 선형회귀모델에서 점들을 가장 잘 나타내는 선을 그었을 때 - 선의 y절편
* eps_ttm의 계수
  * Intercept 값에서와 같은 선의 기울기

In [ ]:
b1, b0 = model.params["eps_ttm"], model.params["Intercept"]

* 절편 = b0
* 기울기 = b1

#### 산점도에 선 추가하기

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
x = price_earning["eps_ttm"]
y = price_earning["close_mean"]
# scatter plot 그리기
ax.scatter(x, y)
# 선형회귀모델을 나타내는 선(regression line) 그리기
ax.plot(x, b1 * x + b0)
ax.set(xlabel='EPS', ylabel='Price')
plt.show()

산점도에 선 추가하기
1. ax.scatter()와 같이 호출에 산점도를 그리고
2. 같은 axe 객체에 plot 호출

-> **같은 axe 객체에 여러 개 도표 추가**하면
<br>
-> 그 도표들이 **모두 겹쳐 표시**됨

### 구간interval을 선 그래프로 그리기

구간을 제공 = 더 풍부한 정보 제공
* 선형 회귀 모델은 응답변수 평균 response mean에 대한 추정값 외에도
  * 응답변수 평균에 대한 신뢰구간 confidence interval과
  * 개별 관측값 observation에 대한 예측구간 prediction interval 추정 가능

-> 이와 같은 구간interval을 선 그래프로 나타내기:

#### get_prediction() 함수로 interval 값들 구하기

* alpha 값에 따라 100(1-alpha) 퍼센트의 interval 구하게 됨
  * alpha=0.05 -> 95%의 신뢰구간 및 예측구간 계산

In [ ]:
prediction = (model
              .get_prediction(price_earning)
              .summary_frame(alpha=0.05))
prediction

각 열에 대한 설명:
* mean: 응답변수 평균의 추정치
* mean_se: 응답변수 평균의 표준오차 standard error
* mean_ci_lower: 신뢰구간 하한
* mean_ci_upper: 신뢰구간 상한
* obs_ci_lower: 예측구간 하한
* obs_ci_upper: 예측구간 상한

#### 각 interval의 상한과 하한을 plot() 함수 이용해 선으로 추가하기

In [ ]:
price_earning["eps_ttm"].mean(), price_earning["close_mean"].mean()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
x = price_earning["eps_ttm"]
y = price_earning["close_mean"]

ax.scatter(x, y)

ax.plot(x, b1 * x + b0)
ax.plot(x, prediction["mean_ci_lower"], c="green",
        linestyle="dashed")
ax.plot(x, prediction["mean_ci_upper"], c="green",
        linestyle="dashed")
ax.plot(x, prediction["obs_ci_upper"], c="red", linestyle="dotted")
ax.plot(x, prediction["obs_ci_lower"], c="red", linestyle="dotted")

ax.axhline(np.mean(y), c="black", linestyle="dashdot")
ax.axvline(np.mean(x), c="black", linestyle="dashdot")

ax.set(xlabel='EPS', ylabel="Price")

plt.show()

## 막대 그래프

### bar(): 막대 그래프 그리기

* 앞의 경우와 같으나 plot()을 bar() 함수로만 변경

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 7))

ax1.bar(price_earning["month"], price_earning["close_mean"],
        label="price", color="blue", alpha=0.3)

ax1.set_xlabel("month")
ax1.set_ylabel("price")
ax1.set_title("Google Price and EPS")
ax1.set_xticks(price_earning["month"][np.arange(0, len(price_earning), step=3)])

ax2 = ax1.twinx()

ax2.bar(price_earning["month"], price_earning["eps_ttm"],
        label="EPS", color="red", alpha=0.3)

ax2.set_ylabel("EPS")
ax2.set_xticks(price_earning["month"][np.arange(0, len(price_earning), step=3)])

fig.legend(loc="upper right")

plt.show()

* 두 axe의 막대가 겹치므로 alpha=0.3과 같이 투명도 부여
* 두 막대 그래프가 겹치는 부분 = 보라색

## 히스토그램

* x축 = 데이터 범위를 나타내는 bin 표시
* y축 = 각각의 bin에 속한 데이터의 빈도(개) 수

-> 데이터가 어떤 분포를 따르는지 시각적으로 파악하기 용이함

* 데이터 분포 파악에 중요 역할: 데이터가 어떤 분포를 따르는지 시각적으로 파악하기 위해 사용
  * ex) 히스토그램 모양이 대체로 종 모양 bell shape 형태라면 해당 데이터는 정규분포를 따른다고 생각할 수 있음
* 다만 **히스토그램 모양 = bin의 수에 따라 판이하게 달라지는 경우**가 있으므로
  * 데이터 분포 확인 위해 다른 방법들을 함께 사용하는 것이 바람직

* 앞서 구글의 주가 데이터로 선형 모델을 생성하고
* residual 값들이 정규분포를 따르는지 정규 확률 그림 normal probability plot으로 확인했으나

-> 히스토그램을 그려 확인하기

### 모델로부터 residual 값들 얻어 내기

In [ ]:
residuals = model.resid

### hist(): 히스토그램 그리기

* residual 값들의 히스토그램 그리기

In [ ]:
fig, ax = plt.subplots()
ax.hist(residuals, bins=7)
plt.show()

* bins 파라미터: 데이터 범위를 몇 개 구간으로 나눠 빈도 표시할지 지정
* 만약 residual 값의 수가 매우 적다면 -> 종 모양에서 크게 벗어나지 않음(교재 참조)

In [ ]:
fig, ax = plt.subplots()
ax.hist(residuals, bins=8)
plt.show()

* 히스토그램 모양이 두 개의 모드mode 갖는 형태

### 히스토그램 해석에 주의

* 히스토그램 모양 = bin 수 요소에 의해 달라질 수 있음
* 데이터 및 bin 수가 작을 때 이와 같은 문제가 더 크고
* 샘플 수가 많은 경우 bin 값을 크게 주면 이와 같은 부분이 상대적으로 덜 문제됨

### 2D 히스토그램: 두 개 값의 조합에 대한 빈도 표현

* 위와 같이 하나의 값에 대한 빈도가 아닌
* 두 개의 값의 조합에 대한 빈도 표현

#### 데이터 불러오기

In [ ]:
google = si.get_data("googl")
fb = si.get_data("meta")
google = google[(google.index > "2020-01-01") &
                (google.index < "2022-01-01")]
fb = fb[(fb.index > "2020-01-01") & (fb.index < "2022-01-01")]

#### 산점도 그리기

* 구글과 페이스북의 종가 데이터로 산점도 그려보기

In [ ]:
fig, ax = plt.subplots()
ax.scatter(google["close"], fb["close"])
plt.show()

* 샘플 수 많아 점들이 겹쳐 표시됨 = 빈도를 정확히 알기 어려움

-> 2D 히스토그램으로 나타내면 분포를 파악하기 조금 더 용이함

#### hist2d(): 2D 히스토그램 그리기

In [ ]:
fig, ax = plt.subplots()
ax.hist2d(google["close"], fb["close"], bins=25, cmap='Blues')
ax.set(xlabel='Google Price', ylabel='Facebook Price',
       title='2D Histogram')
plt.show()

* 인자로 받은 두 값 = x, y 값으로 표현
* x, y 빈도 = 해당 bin의 색상
  * 색이 진할수록 해당 bin의 빈도 높음

-> 산점도로 그렸을 때보다 빈도 파악 용이함

### ⭐⭐ 상자 그림

* box plot
* 여러 데이터의 범위를 한 번에 비교 가능

* 데이터 사분위수를 볼 수 있고
* 이상치 outlier 식별에 좋음
  * 이상치:
    * 원래 모집단이 따르는 분포에서 나온 것으로 보이지 않는 - 특이한,
    * 또는 데이터 관측 과정에서 문제 발생해 다른 값들과 다르게 측정된 것으로 의심되는 값
* 여러 데이터의 범위를 한 번에 비교하기 용이
* 상자 수염 그림 box-and-whisker diagram이라고도 함

* 히스토그램, 상자 그림, 줄기-잎 그림 모두 데이터 분포를 보기 위해 사용
* 탐색적 데이터 분석 EDA(Exploratory Data Analysis)에서
  * 상자 그림 = 데이터 분포 보기 위해 주로 사용
  * 줄기-잎 그림 = 데이터 수가 적을 때 유용

### 백분위수와 사분위수의 이해

#### Order Statistics 순서 통계량

샘플을 작은 값 -> 큰 값 순서로 정렬해 얻어지는 statistics
* e.g. 샘플: 3, 4, 5, 1, 10, 8, 11
* first order statistic y1 = 1, rank 1
* second order statistic y2 = 3, rank 2
* 7th order statistic y7 = 11, rank 7

백분위수 구할 때 사용

#### Sample Percentile 백분위수

(100p)th sample percentile (0 < p < 1)
* 주어진 샘플에서 np개의 같거나 작은 관측값들을 가지는 값
  * n = 샘플의 크기
* (n+1)pth order statistic
  * e.g. 1, 3, 4, 5, 8, 10, 11
    * 50% percentile: y(7+1)0.5 = y4 = 5
    * 50% 백분위수: 그 수보다 작거나 같은 값이 전체 샘플의 50% 만큼 존재

* 위는 대략적 정의로
* 정확한 백분위수를 결정하는 몇 가지 방법 중 하나임

(100p)th sample percentile (0 < p < 1)
* (n+1)p가 소수인 경우 정수부 k와 소수부 r의 합으로 나타낼 수 있다면
  * ̃πₚ=𝑦ₖ+r(yk+1 - yk)
* e.g. 1, 3, 4, 5, 9, 10, 11, 12
  * 50% percentile: (8 + 1)0.5 = 4.5
  * ̃π0.5 = y4 + 0.5(y5 - y4) = 5 + 0.5(9 - 5) = 7

#### Quartile 사분위수

* 50th percentile: median, second quartile, Q2, 2사분위수
* 25th percentile: first quartile, Q1, 1사분위수
* 75th percentile: third quartile, Q3, 3사분위수
* IQR(Interquartile Range): Q3 - Q1
  * 상자 그림에서 상자의 높이 결정
* e.g. 1, 3, 4, 5, 8, 10, 11
  * Q1 = 3, Q2 = 5, Q3 = 10
  * IQR = 10 - 3 = 7

### boxplot(): 상자 그림 그리기

* 데이터가 한 종류
  * 인자 = 리스트, 일차원 배열, Series 등
* 두 종류 이상 데이터 한꺼번에 비교
  * 인자 = 일차원 데이터들을 리스트로 묶어 사용

* 구글의 2017 ~ 2021년까지 종가 데이터를 각 연도별로 상자 그림 그려 비교

#### 데이터 불러오기

In [ ]:
google = si.get_data("googl")

google_2017 = google[(google.index > "2017-01-01") &
                     (google.index < "2018-01-01")]["close"]
google_2018 = google[(google.index > "2018-01-01") &
                     (google.index < "2019-01-01")]["close"]
google_2019 = google[(google.index > "2019-01-01") &
                     (google.index < "2020-01-01")]["close"]
google_2020 = google[(google.index > "2020-01-01") &
                     (google.index < "2021-01-01")]["close"]
google_2021 = google[(google.index > "2021-01-01") &
                     (google.index < "2022-01-01")]["close"]

#### 상자 그림 그리기

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.boxplot([google_2017, google_2018, google_2019,
            google_2020, google_2021])
ax.set(xlabel='year', ylabel='price', title='Google')
ax.set_xticklabels(["2017", "2018", "2019", "2020", "2021"])
plt.show()

상자 그림 = 5개 값 표시
* 상자 중앙 붉은색 선 = 중간값median (Q2)
* 상자 하단, 상단 선 = 데이터를 크기순으로 줄 세울 시 각각 25%, 75% 순서에 해당하는 사분위값 interquartile range
* 상자의 가로 길이는 아무 의미 없음
* 상자 그림 맨 위, 아래에 수평하게 그어진 수염whisker = 일반적으로 데이터 최대값, 최소값
  * = 2017 ~ 2019년 데이터
  * 2020년 데이터: 최대값, 최소값이 상자에서 IQR(상자 높이)×1.5 넘는 구간까지 존재하는 경우
    * IQR×1.5만큼 상자에서 떨어진 곳에 그어지고
    * 해당 범위 바깥의 데이터들은 점으로 표시

## 히트맵

* 두 변수 사이 관계를 값과 함께 색상으로 표현해 시각적으로 도움 주는 그래프
* 앞서 생성했던 correlation 값들로 히트맵 그리기

### 데이터 불러오기

In [ ]:
price_earning = pd.read_csv("price_earning.csv")
corr = price_earning[["eps_ttm", "eps_ttm_diff",
                      "epssurprisepct", "close_mean"]].corr()
corr

### sns.heatmap(): 히트맵 그리기

* 히트맵의 경우 Matplotlib으로 그리는 것이 직관적이지 않으므로
* 일반적으로 seaborn 패키지 사용

* Matplotlib = 파이썬의 기본적 그래프 위한 도구
* seaborn = 보다 복잡하고 세련된 도표들을 간단한 코드로 생성

In [ ]:
import seaborn as sns

sns.heatmap(corr, cmap="YlGnBu", annot=True)
plt.show()

* correlation이 큰 값일수록 진한 색으로 표시
  * 어떤 변수들 사이 correlation이 큰지 한 눈에 확인 가능

인수
* cmap
  * 어떤 색 계열의 히트맵 그릴지 지정
* annot=True
  * 각 셀 안에 값들 표시하는 옵션

## ⭐ 줄기-잎 그림

* stem-and-leaf diagram
* 값들의 분포에 대해 빠르게 훑어볼 수 있게 해 주는 도구 중 하나
* 앞서 본 그래프들과 달리 어느 정도 표에 가까운 형태이나
* 데이터의 직관적 파악에 사용 가능한 도구
* 그림과 표의 중간에 가까운 형태

히스토그램과 차이:
* 줄기-잎 그림: 분포, 데이터값 동시에 표시
* 히스토그램: 분포만 표시

### Matplotlib으로 그릴 수 있는 줄기-잎 그림

* 데이터에서 줄기에 해당하는 부분을 따로 생성해 넣어야 그릴 수 있음

price_earning 데이터 프레임의 eps_ttm_diff 열로 줄기-잎 그림 그리기
* 정수 부분 = 줄기
* 소수 부분 = 잎
1. 수를 인자로 받아 정수 부분을 리턴하는 함수를 map으로 적용해
2. 데이터의 정수 부분만 추린 결과를 stem(줄기) 값으로 저장

In [ ]:
eps_ttm_diff = price_earning["eps_ttm_diff"]
stem = eps_ttm_diff.map(lambda x: int(x))

Matplotlib의 stem() 함수에 인자 전달:
* 줄기 부분 데이터
* 원본 데이터

-> 줄기-잎 그림 생성:
* x축 = 줄기에 해당하는 값
* 점 = 각각의 샘플

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.stem(stem, eps_ttm_diff, use_line_collection=True)
ax.set(xlabel='Stem', ylabel='Value', title='EPS TTM Diff')
ax.set_xticks(np.arange(int(eps_ttm_diff.min()),
                        int(eps_ttm_diff.max()) + 1))
plt.show()

* 이와 같은 Matplotlib의 표현 방식은 일반적 줄기-잎 그림과 다르므로
* 보다 일반적 형태의 줄기-잎 그림 그리기 위해 다른 패키지 사용

### stemgraphic 패키지로 줄기-잎 그림 그리기

#### 설치 및 로드

In [ ]:
pip install stemgraphic

In [ ]:
import stemgraphic

* Matplotlib와 같이 stem 부분을 따로 만들어 줄 필요 없음
* 줄기와 잎 구분 위해 scale 인자 사용
  * scale=1
    * 1의 배수 = 줄기
    * 소수 첫째 자리 = 잎

In [ ]:
fig, ax = stemgraphic.stem_graphic(
    price_earning["eps_ttm_diff"], scale=1)
plt.show()

* 9 | 7 = 9.7
  * 9 | 7 왼쪽의 20 = 아래에서부터 20번째로 큰 값임 (누적계수)
* 3 | 15 = 3.1, 3.5
  * 3 | 15 왼쪽의 18 = 3.5가 18번째로 큰 값임
* 줄기-잎 그림 하단, 상단 숫자 = 최소값, 최대값

In [ ]:
fig, ax = stemgraphic.stem_graphic(
    price_earning["eps_ttm"], scale=10)
plt.show()

eps_ttm 열에 대한 줄기- 잎 그림
* 값들 범위가 넓으므로 scale=1과 같이 파라미터를 주는 것이 의미 없을 수 있음
* scale=10
  * 10의 배수 부분 = 줄기
  * 10보다 작은 부분 = 잎

## 오차 막대 그래프

* 값과 오차를 같이 나타내기 위한 도표
* Matplotlib의 errorbar() 함수로 그릴 수 있음

정보 전달 측면에서
* 추정 = 특정 값 하나로 주어지는 것보다
* 추정값과 오차error가 함께 주어지면

-> 그 추정의 정확성을 같이 고려할 수 있음
<br>
-> 더 풍부한 정보 주어짐

### 앞에서 생성했던 선형 모델로 예측값 만들기

* 앞서 생성했던 모델 = eps_ttm 값으로 주가 예측하는 모델

-> 선형 모델의 예측값 및 신뢰 구간을 오차 막대 그래프로 표현하기

In [ ]:
price_earning["eps_ttm"].mean()

  * 모델 생성 시 사용된 데이터의 eps_ttm 값 평균

In [ ]:
price_earning["eps_ttm"].min(), price_earning["eps_ttm"].max()

* 모델 생성 시 사용된 데이터의 eps_ttm 값 범위

In [ ]:
import random

random.seed(8)

random_eps_ttm = [random.uniform(price_earning["eps_ttm"].min() - 20,
                                 price_earning["eps_ttm"].max() + 20)
                  for i in range(10)]

random_eps_ttm = sorted(random_eps_ttm, reverse=False)

random_eps_ttm

* 모델 생성 시 사용된 데이터의 eps_ttm 값 범위에서 최소-20 ~ 최대+20 범위 랜덤 값들 10개 생성 후
* 오름차순으로 정렬해 random_eps_ttm 변수에 저장

In [ ]:
new_observation = pd.DataFrame({"eps_ttm": random_eps_ttm})

모델 생성 시 사용한 데이터에서 예측변수predictor에 해당하는 열 = eps_ttm

-> 같은 열 이름 갖는 데이터 프레임 생성

In [ ]:
prediction = (model
              .get_prediction(new_observation)
              .summary_frame(alpha=0.05))
prediction

모델의 get_prediction 함수에 데이터 프레임 넣어 예측구간 prediction interval 얻음

In [ ]:
prediction["prediction_error"] = (
    prediction["mean"] - prediction["obs_ci_lower"])
prediction["prediction_error"]

오차 막대 그리기 위해 필요한 prediction 오차값

= 응답변수 평균 추정치 - prediction interval 하한

### errorbar(): 오차 막대 그래프 그리기

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
ax.errorbar(x = new_observation["eps_ttm"],
            y = prediction["mean"],
            yerr = prediction["prediction_error"],
            fmt='o', markersize=5, capsize=5)
ax.set(xlabel='EPS TTM', ylabel='Price',
       title='Prediction Interval')
plt.show()

인수
* x = new_observation["eps_ttm"]
* y = prediction["mean"] = x에 대한 응답변수 평균 추정치
* yerr = prediction["prediction_error"]
* fmt = 그래프 포맷 지정
* markersize = 오차 막대 가운데 점 크기 지정
* capsize = 오차 막대 끝의 수평선 길이 지정

* 모델 생성 시 사용된 price_earning 데이터 프레임의 eps_ttm 평균에 가까운 eps_ttm 값에 대한 예측에서는 오차 크기 작음
  * 평균에서 멀어지면 오차 커짐
  * 특히 모델 생성에 사용된 데이터의 eps_ttm 값 범위를 벗어나는 값에 대한 예측에서는 오차 크기 급격히 커짐

-> 모델 생성에 사용된 데이터에서는 x와 y 사이에 선형적 관계가 성립해도

-> 그 범위 밖에서 선형적 관계가 성립할지 알 수 없기 때문

-> **모델 생성에 사용된 데이터 범위 바깥 값 예측에 선형 모델 사용 시 주의 필요**